In [4]:
import numpy as np
import copy
from collections import namedtuple
from tabulate import tabulate

In [5]:
CotesMethod = namedtuple('CotesMethod', 'numerators denominator')
left_rectangle = CotesMethod([1, 0], None)
right_rectangle = CotesMethod([0, 1], None)
middle_rectangle = CotesMethod([0, 1, 0], None)
trapezium = CotesMethod([1, 1], 2)
simpson = CotesMethod([1, 4, 1], 6)
cotes3 = CotesMethod([1, 3, 3, 1], 8)
cotes4 = CotesMethod([7, 32, 12, 32, 7], 90)
cotes5 = CotesMethod([19, 75, 50, 50, 75, 19], 288)
cotes6 = CotesMethod([41, 216, 27, 272, 27, 216, 41], 840)

def cotes_integration(function, method, a, b, n):
    
    result = 0
    m = len(method.numerators)
    
    h = (b - a) / n
    for i in range(n):
        for j in range(m):
            if method.numerators[j] != 0:
                x = a + (b - a) * (i + j/(m-1)) / n 
                f = function(x) * method.numerators[j]
                
                result += f * h 
        
    if not method.denominator is None:
        result /= method.denominator
        
    return result;

def monte_carlo_integration(function, a, b, n):
    x = np.random.rand(n) * (b - a) + a
    return (b - a) * function(x).sum() / n

def geometric_monte_carlo_integration(function, a, b, n):
    x = np.random.rand(n) * (b - a) + a
    f = function(x)
    M = np.abs(f).max()
    y = np.random.rand(n) * 2 * M - M
    result = (((f > y) & (y > 0)).sum() - ((f < y) & (y < 0)).sum()) * 2 * M * (b - a) / n
    
    return result

In [6]:
a = 0
b = 1

exp = lambda x: np.exp(- x**2)
square = lambda x: x ** 2
cubic = lambda x: x ** 3

functions = {
    'left_rectangle' : lambda f, a, b, n: cotes_integration(exp, left_rectangle, a, b, n),
    'right_rectangle' : lambda f, a, b, n: cotes_integration(exp, right_rectangle, a, b, n), 
    'middle_rectangle' : lambda f, a, b, n: cotes_integration(exp, middle_rectangle, a, b, n), 
    'trapezium' : lambda f, a, b, n: cotes_integration(exp, trapezium, a, b, n), 
    'simpson' : lambda f, a, b, n: cotes_integration(exp, simpson, a, b, n), 
    'cotes3' : lambda f, a, b, n: cotes_integration(exp, cotes3, a, b, n),
    'cotes4' : lambda f, a, b, n: cotes_integration(exp, cotes4, a, b, n), 
    'cotes5' : lambda f, a, b, n: cotes_integration(exp, cotes5, a, b, n),
    'cotes6' : lambda f, a, b, n: cotes_integration(exp, cotes6, a, b, n),
    'monte_carlo': lambda f, a, b, n: monte_carlo_integration(exp, a, b, n),
    'geometric_monte_carlo': lambda f, a, b, n: geometric_monte_carlo_integration(exp, a, b, n)
}

origin = 0.746824132812427025399

for n in [10, 100, 1000]:
    results = []
    for name, function in functions.items():
        value = function(exp, a, b, n)
        error = abs(origin - value)
        results.append((name, value, error))

    results.sort(key=lambda p: p[2])
    print("N: {}".format(n))
    print(tabulate(results, headers=["name", "value", "error"], floatfmt=".13f", tablefmt="psql"))

N: 10
+-----------------------+-----------------+-----------------+
| name                  |           value |           error |
|-----------------------+-----------------+-----------------|
| cotes6                | 0.7468241328124 | 0.0000000000000 |
| cotes5                | 0.7468241328133 | 0.0000000000009 |
| cotes4                | 0.7468241328140 | 0.0000000000015 |
| cotes3                | 0.7468241555088 | 0.0000000226964 |
| simpson               | 0.7468241838759 | 0.0000000510635 |
| middle_rectangle      | 0.7471308777480 | 0.0003067449356 |
| trapezium             | 0.7462107961317 | 0.0006133366807 |
| monte_carlo           | 0.7769204155753 | 0.0300962827629 |
| left_rectangle        | 0.7778168240732 | 0.0309926912608 |
| right_rectangle       | 0.7146047681903 | 0.0322193646221 |
| geometric_monte_carlo | 0.9982641530186 | 0.2514400202062 |
+-----------------------+-----------------+-----------------+
N: 100
+-----------------------+-----------------+--------------